In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.api as sm
%matplotlib inline
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt   
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [2]:
df=pd.read_csv('insurance.csv')

In [3]:
df1=df.copy()

In [4]:
df1.head()

,age,gender,bmi,children,smoker,region,claim
0,19,female,27.9,0,1,southwest,16884.92
1,18,male,33.8,1,0,southeast,1725.55
2,28,male,33.0,3,0,southeast,4449.46
3,33,male,22.7,0,0,northwest,21984.47
4,32,male,28.9,0,0,northwest,3866.86


In [5]:
df1=pd.get_dummies(df1,drop_first=True)

In [6]:
df1.head()

,age,bmi,children,smoker,claim,gender_male,region_northwest,region_southeast,region_southwest
0,19,27.9,0,1,16884.92,0,0,0,1
1,18,33.8,1,0,1725.55,1,0,1,0
2,28,33.0,3,0,4449.46,1,0,1,0
3,33,22.7,0,0,21984.47,1,1,0,0
4,32,28.9,0,0,3866.86,1,1,0,0


In [7]:
X=df1.drop(columns={'claim'})
y=df1['claim']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.3,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(936, 8)
(402, 8)
(936,)
(402,)


In [9]:
import statsmodels.api as sm
X_constant=sm.add_constant(X)
model=sm.OLS(y,X_constant).fit()
model.summary()

/opt/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  claim   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.9
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        13:19:09   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.194e+04    987.811    -12.089      0.000   -1.39e+04      -1e+04
age                256.8392     11.899     21.586      0.000     233.497     280.181
bmi                339.2899     28.598     11.864      0.000     283.187     395.393
children           475.6889    137.800      3.452      0.001     205.360     746.017
smoker            2.385e+04    413.139     57.723      0.000     2.3e+04    2.47e+04
gender_male       -131.3520    332.935     -0.395      0.693    -784.488     521.784
region_northwest  -352.7901    476.261     -0.741      0.459   -1287.095     581.515
region_southeast -1035.5957    478.681     -2.163      0.031   -1974.648     -96.544
region_southwest  -959.3058    477.912     -2.007      0.045   -1896.850     -21.762
==============================================================================
Omnibus:                      300.499   Durbin-Watson:                   2.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              719.382
Skew:                           1.212   Prob(JB):                    6.14e-157
Kurtosis:                       5.652   Cond. No.                         311.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
model.pvalues

const               5.389279e-32
age                 7.908383e-89
bmi                 6.234283e-31
children            5.738428e-04
smoker              0.000000e+00
gender_male         6.932550e-01
region_northwest    4.589762e-01
region_southeast    3.068532e-02
region_southwest    4.492142e-02
dtype: float64

All the variables are given as input and eventually we remove the features one by one which is not creating impact to the model
Performance metric used here to evaluate feature performance is pvalue. If the pvalue is above 0.05 then we remove the feature

In [11]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)   #it is iterating each time untill p<0.05 from entire model   
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['age', 'bmi', 'children', 'smoker']


# 2.2. RFE - Recursive Feature Elimination

In [12]:
model = LinearRegression()

In [13]:
#Initializing RFE model
rfe = RFE(model, 11)

In [14]:
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1]


##### RFE (model , num) :: Here num represents the number of features we want to include as our model building step.
##### As here we are giving 11 so it will assign 11 features as 1 , i.e. it will find 11 most important features.

In [15]:
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1]


In [16]:
X.columns

Index([u'age', u'bmi', u'children', u'smoker', u'gender_male',
       u'region_northwest', u'region_southeast', u'region_southwest'],
      dtype='object')

##### From above two steps , we can see that it has assigned 1 to three featues :: They are CRIM, ZN,INDUS,CHAS,NOX,RM, DIS, RAD, TAX, PTRATIO, LSTAT. It is upto us, we can take any number of features for our model building.

In [17]:
#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test) #store the R score
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 8
Score with 8 features: 0.790958


In [18]:
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 8)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index([u'age', u'bmi', u'children', u'smoker', u'gender_male',
       u'region_northwest', u'region_southeast', u'region_southwest'],
      dtype='object')


# 2.1. Step Forward Selection

In [19]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [20]:
import warnings
warnings.filterwarnings('ignore')

Note randomly the value of k_features is taken as 10, we can take any value < total number of features. But remember that it should not be too low, as becuase if the value of features is too low, then it can pollute the subset.

In [22]:
# Build RF classifier to use in feature selection
clf = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)


# Build step forward feature selection
sfs1 = sfs(clf,k_features = 8,forward=True,
           floating=False, scoring='r2',
           verbose=2,
           cv=5)# cv is number of iteration for each variable

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s finished

[2019-12-04 13:20:53] Features: 1/8 -- score: 0.5859099389877302[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s finished

[2019-12-04 13:20:53] Features: 2/8 -- score: 0.6927770536378232[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished

[2019-12-04 13:20:53] Features: 3/8 -- score: 0.7206664307633659[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

# So, we can see that For  8 (8 out of 8 features) , the r2 (R_squared value) is maximum i.e. 72.8%. So, we will build the model again with K_features = 10.

In [24]:
# Build RF classifier to use in feature selection
clf = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)


# Build step forward feature selection
sfs1 = sfs(clf,k_features = 8,forward=True,
           floating=False, scoring='r2',
           verbose=2, 
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished

[2019-12-04 13:22:28] Features: 1/8 -- score: 0.5859099389877302[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s finished

[2019-12-04 13:22:28] Features: 2/8 -- score: 0.6927770536378232[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished

[2019-12-04 13:22:28] Features: 3/8 -- score: 0.7206664307633659[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [25]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4, 5, 6, 7]


In [26]:
X.columns

Index([u'age', u'bmi', u'children', u'smoker', u'gender_male',
       u'region_northwest', u'region_southeast', u'region_southwest'],
      dtype='object')